## Sphere

In [ ]:
LEN = 5
A = np.zeros((LEN,LEN,LEN))

center_x = 0.5*(LEN-1)
center_y = 0.5*(LEN-1)
center_z = 0.5*(LEN-1)
Radius = 2
for xx in range(LEN):
    for yy in range(LEN):
        for zz in range(LEN):
            dist = np.sqrt((xx - center_x)**2 + (yy - center_y)**2 + (zz - center_z)**2)
            if dist <= Radius:
                A[xx, yy, zz] = 1
print(A)
print(np.sum(A))
print(center_x)
plt.imshow(A[np.int(center_x),:,:])

## Create square and circle around chosen point

In [ ]:
# start with a 100 x 100 square grid
Nside = 100
base_indices = np.arange(Nside)
x_indices, y_indices = np.meshgrid(base_indices, base_indices)
reference_position_indices = np.array([37, 20])
# disk = np.where(np.sqrt((x_indices - reference_position_indices[0])**2 + (y_indices - reference_position_indices[1])**2)<= 10, 1, 0)
np.sqrt((x_indices - reference_position_indices[0])**2 + (y_indices - reference_position_indices[1])**2)<= 10
plt.imshow(disk)

In [ ]:
square = np.where(np.logical_and(np.abs(x_indices - reference_position_indices[0]) <= 5, np.abs(y_indices - reference_position_indices[1]) <= 5), 1, 0)
plt.imshow(square)

## np.where will ravel an array with any shape to a 1D array

In [ ]:
x = np.arange(100).reshape(10,10)
print(x)
x[3:7,1:5] = np.where(x[3:7,1:5] % 2, 0, x[3:7,1:5])
print(x)

## Slicing array with a condition

In [ ]:
x = np.arange(20)
avg_1 = x[np.logical_and(7 <= x, x <= 14)].mean()
avg_2 = np.where(np.logical_and(7 <= x, x <= 14), x, 0).mean()
avg_1 == avg_2

In [ ]:
x_values = np.arange(20)
condition = np.logical_and(7 <= x_values, x_values <= 14)
x_values[condition]

## Averaging sphere

In [ ]:
test_box = np.random.rand(101,101,101)
test_center = [0.5*(np.shape(test_box)[0]-1), 0.5*(np.shape(test_box)[1]-1), 0.5*(np.shape(test_box)[2]-1)]

true_false_index_x = np.arange(np.shape(test_box)[0])
true_false_index_y = np.arange(np.shape(test_box)[1])
true_false_index_z = np.arange(np.shape(test_box)[2])
test_x, test_y, test_z = np.meshgrid(true_false_index_x, true_false_index_y, true_false_index_z)

circle_condition = np.sqrt((test_x - test_center[0])**2 + (test_y - test_center[1])**2 + (test_z - test_center[2])**2) <= 30
inside_circle_mean = np.mean(test_box[circle_condition])
test_box = np.where(circle_condition, inside_circle_mean, test_box)

plt.imshow(test_box[:,:,21])

## Light cone: Global Average Neutral Fraction vs. Redshift

In [ ]:
lightcone = p21c.run_lightcone(
    redshift = 6,
    max_redshift = 8,
    init_box=init_cond,
    lightcone_quantities=(
        'brightness_temp',
        'xH_box',
    ),
    global_quantities=(
        'brightness_temp',
        'xH_box'
    )
)

p21c.plotting.plot_global_history(lightcone, kind='xH_box')
plt.title("Neutral Freaction vs. Redshift")

## Plotting slices of Neutral fractions.

In [ ]:
def simple_xH_plot(ion: p21c.outputs.IonizedBox, ax: plt.Axes):
    return ax.imshow(ion.xH_box[:, :, 0].T, extent=(0,200,0,200), vmin=0, vmax=1, origin='lower')

In [ ]:
fig, ax = plt.subplots(2, 4, figsize=(14,6), sharex=True, sharey=True, f)

for i, z in enumerate([20, 12, 10, 9, 8, 7, 6, 5]):
    ion = p21c.ionize_box(redshift=z, init_boxes=init_cond)
    im = simple_xH_plot(ion, ax.flatten()[i])
    ax.flatten()[i].set_title(f"z = {z}, Age = {cosmo_params.cosmo.age(z):.2f}")
    
cbar = plt.colorbar(im, ax=ax.ravel().tolist())
cbar.set_label("Neutral Fraction")

## unblurred and blurred side by side comparison

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(15,15))

img1 = ax1.imshow(ionized[:, :, 0], extent=(0,BOX_LEN,0,BOX_LEN), origin='lower')
ax1.set_title(f"z = {z}, Age = {cosmo_params.cosmo.age(z):.2f}", fontsize=16)
ax1.set_ylabel("y [voxel]")
ax1.set_xlabel("x [voxel]")
colorbar(img1)


img2 = ax2.imshow(Sphere_Blurring(ionized, 140, 1)[:, :, 0], extent=(0,BOX_LEN,0,BOX_LEN), origin='lower')
ax2.set_title(f"Blurred, z = {z}, Age = {cosmo_params.cosmo.age(z):.2f}", fontsize=16)
ax2.set_xlabel("x [voxel]")
colorbar(img2)

plt.tight_layout(h_pad=1)

## x, y, z slices of the blurred box

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(ncols=3, figsize=(15,15), dpi=200)

img1 = ax1.imshow(Sphere_blurred[0,:,:], origin='lower')
colorbar(img1)

img2 = ax2.imshow(Sphere_blurred[:,0,:], origin='lower')
colorbar(img2)

img3 = ax3.imshow(Sphere_blurred[:,:,0], origin='lower')
colorbar(img3)

plt.tight_layout(h_pad=1)

## Generator

In [ ]:
shape = (300, 100, 2740, 15, 22)
list(np.random.randint(0,side_length) for side_length in shape)
# generator = [expression for item(s) in iterable if condition]

## Calculate Average with big function broken up into smaller functions

In [ ]:
def calculate_average(box, radius, reference_position, shape='cube', weights=None):
    """
    Given an array ``box``, calculate the average over some region with radius
    ``radius`` centered on location ``reference_position``.
    """
    # define a smaller cube with sides 2*radius+1
    # condition for sphere
    # mean(box[condition_for_sphere])
    
    # retrieve the region of interest
    # region_of_interest = get_subarray(box, radius, reference_position, shape)
    # mean of region and return value
    # weights = generate_weights(region_of_interest, weights) # this is extra; don't worry for now
    # return np.average(region_of_interest, weights=weights)

## Definition of Weighted Average

In [1]:
# weighted_average = sum(weights * array) / sum(weights)

## Plotted gaussian and compute area under the curve

In [4]:
def Gaussian(x, sigma=1, mu=0):
    return (1/sigma/np.sqrt(2*np.pi))*(np.exp(-0.5*(((x-mu)/sigma)**2)))

import numpy as np
x = np.linspace(-1.4370396997748922,1.4370396997748922,10**3)
sigma = 1
g = Gaussian(x, sigma=sigma)
# fig = plt.figure(figsize=(15,15))
# plt.plot(x, g, [-4*sigma,-4*sigma], [-0.01, 0.2], [4*sigma,4*sigma], [-0.01, 0.2])

from scipy.integrate import simps

area = simps(g, x)
print(area)

0.849293286864741


## plotting the shells

In [ ]:
radius = 150
shell_box = top_hat_shell_averaging(distance_from_coordinate([radius*2+1]*3), radius, noise_box)

fig, (ax1, ax2, ax3) = plt.subplots(ncols=3, figsize=(15,15))

img1 = ax1.imshow(shell_box[150,:,:], origin='lower')
colorbar(img1)

img2 = ax2.imshow(shell_box[:,150,:], origin='lower')
colorbar(img2)

img3 = ax3.imshow(shell_box[:,:,150], origin='lower')
colorbar(img3)

plt.tight_layout(h_pad=1)

## Average 4 spherical shells in a given box,  with diameter = sides of box, sphere centered on cube's center

In [ ]:
def top_hat_shell_averaging(distance_box, radius, input_box):
    
    sphere_condition =  distance_box <= radius
        
    inside_sphere_mean = np.mean(input_box[sphere_condition])  # slicing shaped array by condition will ravel the array

    shell_condition1 =  distance_box <= 0.25*radius
    shell_condition2 =  np.logical_and(distance_box <= 0.5*radius, distance_box > 0.25*radius)
    shell_condition3 =  np.logical_and(distance_box <= 0.75*radius, distance_box > 0.5*radius)
    shell_condition4 =  np.logical_and(distance_box <= radius, distance_box > 0.75*radius)
    
    output_box = np.where(shell_condition1, inside_sphere_mean, input_box)
    output_box = np.where(shell_condition2, 0.75*inside_sphere_mean, output_box)
    output_box = np.where(shell_condition3, 0.5*inside_sphere_mean, output_box)
    output_box = np.where(shell_condition4, 0.25*inside_sphere_mean, output_box)

    return output_box

## Sphere Blurring Function

In [ ]:
def Sphere_Blurring(box, radius, iteration):
    
    box = box.copy()  # make copy of input box to have a separate box
    
    dist_frm_coord_box = distance_from_coordinate([radius*2+1]*3)

    inside_sphere_mean_data_top_hat = []  # empty list for data collection
    inside_sphere_mean_data_gaussian = []
    
    for i in range(iteration):
        
        rand_coord = random_voxel(box.shape)
                
        cube_region_indices, cube_region_box = cube_centered_about_coordinate(rand_coord, radius, box)
        
        inside_sphere_mean_top_hat, sphere_condition_box = top_hat_sphere_average_value(dist_frm_coord_box, radius, cube_region_box)

        inside_sphere_mean_gaussian, sphere_condition_box = gaussian_sphere_average_value(dist_frm_coord_box, radius, cube_region_box)
        # returning sphere_condition_box only for plotting box purposes, could be removed if plotting the box is no longer required
        
        
        
#         inside_sphere_mean_data_top_hat = np.append(inside_sphere_mean_data_top_hat, inside_sphere_mean_top_hat)
#         inside_sphere_mean_data_gaussian = np.append(inside_sphere_mean_data_gaussian, inside_sphere_mean_gaussian)
#     return np.array(inside_sphere_mean_data_top_hat), np.array(inside_sphere_mean_data_gaussian)


        cube_region_box = np.where(sphere_condition_box, inside_sphere_mean_gaussian, cube_region_box)
        box[cube_region_indices] = cube_region_box
    return box

## Cube Blurring Function

In [ ]:
def Square_Blurring(box, radius, iteration):
    box = box.copy()  # make copy of input box to have a separate box
    for i in range(iteration):
        np.random.seed()  # set seed to a randome number
        
        rand_coord = np.random.randint(radius, BOX_LEN-1-radius, size=3)  # ensure that the random center point won't be at the edge of the cube
        
        cube_region = box[rand_coord[0]-radius:rand_coord[0]+radius+1, rand_coord[1]-radius:rand_coord[1]+radius+1, rand_coord[2]-radius:rand_coord[2]+radius+1]
        
        neutral_mean = np.mean(cube_region)
        
        box[rand_coord[0]-radius:rand_coord[0]+radius+1, rand_coord[1]-radius:rand_coord[1]+radius+1, rand_coord[2]-radius:rand_coord[2]+radius+1] = neutral_mean
    return box

## Computing the sphere radius to gaussian radius ratio

In [1]:
from scipy.integrate import simps
r_g = 1.4
error = 1
while error > 10**-8:
    x = np.linspace(0, r_g, 10**3)
    g = np.exp(-x**2)*x**2
    area = simps(g,x)
    r_g += 10**-8
    error = (1/3) - area
    
print(r_g, area)

fig = plt.figure(figsize=(15,15))
plt.plot(x, g)

## Tuning the radius ratios of the 3 different methods

In [ ]:
var_Gaussian = np.var(gaussian)
var_sphere = np.var(sphere)
var_cube = np.var(cube)

print(var_Gaussian)
print(var_sphere)
print(var_cube)

print(abs((var_Gaussian-var_sphere)/np.mean([var_Gaussian, var_sphere])))
print(abs((var_Gaussian-var_cube)/np.mean([var_Gaussian, var_cube])))
print(abs((var_cube-var_sphere)/np.mean([var_cube, var_sphere])))

## Plotting function single plot condition

In [ ]:
fs = (gaussian1, gaussian2, [1,1,1])
fs2 = np.array(np.array([gaussian1]))
type(fs)
type(fs) is tuple
type(fs2) is tuple

## Making list of titles

In [ ]:
titles = []
for radius in radii:
    titles = np.append(titles, f'Radius = {radius}')
    
print(titles)

## Reshaping 1D array

In [1]:
A = np.arange(1, 4)
B = np.arange(4, 7)
C = np.arange(7, 10)
print(A, B, C)
D = []
for a in A:
    for b in B:
        for c in C:
            D = np.append(D, f'{a, b, c}')
pritnt(D)
D = D.reshape(len(A), len(B), len(C))
D[:,0,:]

NameError: name 'np' is not defined

## Print current time

In [ ]:
import time
time.ctime() # 'Mon Oct 18 13:35:29 2010'
time.strftime('%l:%M%p %Z on %b %d, %Y') # ' 1:36PM EDT on Oct 18, 2010'
time.strftime('%l:%M%p %z on %b %d, %Y') # ' 1:36PM EST on Oct 18, 2010'

In [1]:
import numpy as np
1-np.exp(-1)

0.6321205588285577

## colorbar function

In [ ]:
def colorbar(mappable):
    from mpl_toolkits.axes_grid1 import make_axes_locatable
    import matplotlib.pyplot as plt
    last_axes = plt.gca()
    ax = mappable.axes
    fig = ax.figure
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    cbar = fig.colorbar(mappable, cax=cax, label="Neutral Fraction")
    plt.sca(last_axes)
    return cbar

## Creating Noise Box

In [ ]:
np.random.seed()
# np.random.seed(5) ; np.random.rand(10)
noise_box = np.random.rand(301, 301, 301)

## Creating plots of standard deviation of average neutral fraction distributions as a function of shell numbers and radii.

In [ ]:
def generate_distribution_standard_deviations():

    iterations = 3*10**2
    radii = np.arange(10, 100, 10)  # [10, 20, 30, ... 90]
    sigma_factors = np.arange(1, 6)  # [1, 2, 3, 4, 5]
    shell_numbers = np.arange(1, 9)  # [1, 2, 3, 4, 5, 6, 7, 8]
    stand_devs_gaussian = []  # empty list for data collection
    stand_devs_sphere = []
    stand_devs_cube = []

    for radius in radii:

        sphere = Average_Neutral_Fraction_Distribution(
                box=noise_box,
                radius=radius,
                iteration=iterations,
                blur_shape='top_hat_sphere'
            )

        cube = Average_Neutral_Fraction_Distribution(
                box=noise_box,
                radius=radius,
                iteration=iterations,
                blur_shape='top_hat_cube'
            )

        stand_dev_sphere = np.std(sphere)
        stand_dev_cube = np.std(cube)

        for shell_num in shell_numbers:

            stand_devs_sphere = np.append(stand_devs_sphere, stand_dev_sphere)
            stand_devs_cube = np.append(stand_devs_cube, stand_dev_cube)

            for sigma_factor in sigma_factors:

                gaussian = Average_Neutral_Fraction_Distribution(
                    box=noise_box,
                    radius=radius,
                    sigma_factor=sigma_factor,
                    iteration=iterations,
                    shell_num=shell_num,
                    blur_shape='Gaussian_sphere'
                )

                stand_dev_gaussian = np.std(gaussian)

                stand_devs_gaussian = np.append(stand_devs_gaussian, stand_dev_gaussian)

        print(f'progress = {int(round(radius*10/len(radii)))}%')

    stand_devs_gaussian = stand_devs_gaussian.reshape(len(radii), len(shell_numbers), len(sigma_factors))
    stand_devs_sphere = stand_devs_sphere.reshape(len(radii), len(shell_numbers))
    stand_devs_cube = stand_devs_cube.reshape(len(radii), len(shell_numbers))
    
    return stand_devs_gaussian, stand_devs_sphere, stand_devs_cube

## Plotting Standard Deviations

In [ ]:
stand_devs_gaussian, stand_devs_sphere, stand_devs_cube = generate_distribution_standard_deviations()


titles = []
for radius in radii:
    titles = np.append(titles, f'Radius = {radius}')
    
plots(
    x_vals=shell_numbers, 
    y1s=tuple(stand_devs_gaussian[:,:,0]),
    y2s=tuple(stand_devs_gaussian[:,:,1]),
    y3s=tuple(stand_devs_gaussian[:,:,2]),
    y4s=tuple(stand_devs_gaussian[:,:,3]),
    y5s=tuple(stand_devs_gaussian[:,:,4]),
    y6s=tuple(stand_devs_sphere), 
    y7s=tuple(stand_devs_cube),
    y1s_label='Gaus 1σ',
    y2s_label='Gaus 2σ',
    y3s_label='Gaus 3σ',
    y4s_label='Gaus 4σ',
    y5s_label='Gaus 5σ',
    y6s_label='Sphere', 
    y7s_label='Cube',
    shared_y_label='Standard Deviation', 
    shared_x_label='Number of Weighted Bins / Spherical Shells', 
    figure_shape=(3,3), 
    figure_size=(18,18), 
    title=titles, 
    shared_title=f'Standard Deviation of Average Neutral Fraction Distribution\
    (Radius Ratio 2, Random Unseeded Voxels, iterations = {iterations})',
    vertical_gap=0.1,
    horizontal_gap=0.2,
    y_notation='sci',
    share_y_axis=False,
    figure_resolution=1000  # units: dpi
)